# Imports

In [1]:
from sklearn.utils import shuffle
import pandas as pd
import pickle

# Modeling the text

## Loading data


In [2]:
stage2test_text_df = pd.read_csv("/mnt/4_TB_HD/ramona/utils/stage2_data/stage2_test_text.csv", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
stage2test_variants_df = pd.read_csv("/mnt/4_TB_HD/ramona/utils/stage2_data/stage2_test_variants.csv")
stage2test_class_df = pd.read_csv('/mnt/4_TB_HD/ramona/utils/stage2_data/stage_2_private_solution.csv')

merge training text and variants into one data frame and drop the texts which have no content

In [3]:
stage2test_merge_df = stage2test_variants_df.merge(stage2test_text_df,left_on="ID",right_on="ID")
print(stage2test_merge_df.shape)
stage2test_merge_df.loc[:, 'Text_count'] = stage2test_merge_df["Text"].apply(lambda x: len(str(x).split()) )
stage2test_merge_df = stage2test_merge_df[stage2test_merge_df['Text_count'] != 1]


print(stage2test_merge_df.shape)
stage2test_merge_df.head(5) 



(986, 4)
(986, 5)


,ID,Gene,Variation,Text,Text_count
0,1,CHEK2,H371Y,The incidence of breast cancer is increasing i...,5037
1,2,AXIN2,Truncating Mutations,An unselected series of 310 colorectal carcino...,10245
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...,8549
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...,6168
4,5,BRAF,T599insTT,Pilocytic astrocytoma (PA) is emerging as a tu...,3262


There is no 'null' text

In [4]:
stage2test_merge_df['Text'] = stage2test_merge_df['Text'].apply(lambda x: x.replace('\t',' ')) 

In [5]:
stage2test_merge_df.head()

,ID,Gene,Variation,Text,Text_count
0,1,CHEK2,H371Y,The incidence of breast cancer is increasing i...,5037
1,2,AXIN2,Truncating Mutations,An unselected series of 310 colorectal carcino...,10245
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...,8549
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...,6168
4,5,BRAF,T599insTT,Pilocytic astrocytoma (PA) is emerging as a tu...,3262


In [7]:
def extract_text_sections(Text, Gene, Variation, ID): 
    section=''
    index = Text.find(Variation)
    if index == -1:
      Variation = 'mutation'
      index = Text.find(Variation)
    end_index = 0    
    t = 0
    index_list = []
    while index != -1:
      index_list.append(index)
      old_index = index
      index = Text.find(Variation, old_index+1)
      t+ = 1
    
    
    # for the case the variation appears less the 5 times
    if t < 5 and t>0: 
  
      for index in index_list: 
        # if two sections are overlapping 
        if index <= end_index: 
          # determine end point depending on if it's out of range or not 
          if index+int(2000/t) <= (len(Text)-1): 
            end = index+int(2000/t)
          else: 
            end = (len(Text)-1)
          section += Text[end_index:end]
       
        else: 
          # determine end point depending on if it's out of range or not
          if index+int(2000/t) <=(len(Text)-1): 
            end = index+int(2000/t)
          else: 
            end = (len(Text)-1)
          # determine start point depending on if it's out of range or not
          if index-int(2000/t) >= 0: 
            start = index-int(2000/t)
          else: 
            start = 0
          section += Text[start:end] 
         
  
        end_index = end
    if t==0:
      section = Text
    else: 
       for index in index_list :
        if index <= end_index: 
          # determine end point 
          if index+1000 <= (len(Text)-1): 
            end = index+1000
          else: 
            end = (len(Text)-1)
          section += Text[end_index:end]
       
        else: 
          if index+1000 <= (len(Text)-1): 
            end = index+1000
          else: 
            end = (len(Text)-1)
          if index-1000 >= 0: 
            start = index-1000
          else: 
            start = 0
          section += Text[start:end] 
        
        end_index = end
    section = section.replace('\t', ' ')
    return section
   
   


In [8]:
stage2test_merge_df['text_modified'] = stage2test_merge_df[['Text', 'Gene','Variation', 'ID']].apply(lambda x: extract_text_sections(x[0].lower(), x[1].lower(), x[2].lower(), x[3]),axis=1)

In [9]:
stage2test_merge_df.to_pickle('stage2test_1000.sav')

In [7]:
stage2test_class_df.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,8,0,0,0,1,0,0,0,0,0
1,15,0,0,0,0,0,0,0,0,1
2,16,0,0,0,0,0,0,0,1,0
3,18,0,1,0,0,0,0,0,0,0
4,19,0,1,0,0,0,0,0,0,0


In [33]:
stage2test_class_df.columns = ['ID', 1, 2, 3, 4, 5, 6,
       7, 8, 9]
Class_df = pd.DataFrame()
Class_df['ID'] = stage2test_class_df['ID']
Class_df['Class'] =  stage2test_class_df[[1, 2, 3, 4, 5, 6,
       7, 8, 9]].idxmax(axis=1)

Class_df.head(5) #Takes the info from solution

,ID,Class
0,8,4
1,15,9
2,16,8
3,18,2
4,19,2


In [34]:
stage2test_classified_df = stage2test_merge_df.merge(Class_df,left_on="ID",right_on="ID")


stage2test_classified_df.loc[:, 'Text_count'] = stage2test_classified_df["Text"].apply(lambda x: len(str(x).split()) )
stage2test_classified_df = stage2test_classified_df[stage2test_classified_df['Text_count'] != 1]

print('Test classified'.ljust(15),stage2test_classified_df.shape)
stage2test_classified_df.head(5)

Test classified (125, 6)


,ID,Gene,Variation,Text,Text_count,Class
0,8,RNF6,G244D,Human ESCCs 2 occur frequently worldwide (1) ....,1741,4
1,15,ERBB2,G746S,The protein-kinase family is the most frequent...,11561,9
2,16,TP53,Y234S,Among the best-studied therapeutic targets in ...,15801,8
3,18,EGFR,P546S,Head and neck squamous cell carcinoma (HNSCC) ...,4092,2
4,19,ERBB2,G279E,Functional characterization of cancer-associat...,8273,2


In [35]:
#stage2test_classified_df[['Text'].to_pickle('stage2test_text_only_with_classes.sav')
stage2test_classified_df[['ID','Class']].to_pickle('stage2test_classes.sav')